## 抓取多個城市當月氣象

In [ ]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import time

# 資料欄位名稱，製作df用
columns_t = ['年月',
             '日',
             '1-1氣壓(hPa)-平地平均',
             '1-2氣壓(hPa)-海面平均',
             '2-1降水量(mm)-合計',
             '2-2降水量(mm)-一小時內最大',
             '2-3降水量(mm)-10分鐘內最大',
             '3-1氣温(℃)-平均',
             '3-2氣温(℃)-最高',
             '3-3氣温(℃)-最低',
             '4-1湿度(％)-平均',
             '4-2湿度(％)-最小',
             '5-1風向・風速(m/s)-平均風速',
             '5-2風向・風速(m/s)-最大風速',
             '5-3風向・風速(m/s)-最大風向',
             '5-4風向・風速(m/s)-最大瞬間風速',
             '5-5風向・風速(m/s)-最大瞬間風向',
             '6日照時間(h)',
             '7-1雪(cm)-降雪',
             '7-2雪(cm)-最深積雪',
             '8-1天気概況-晝',
             '8-2天気概況-夜']

# 將數字調整成兩位數

def twobyte(kindno):
    if kindno < 10:
        kindnostr = "0" + str(kindno)
    else:
        kindnostr = str(kindno)
    return kindnostr


# 讀取需抓取的47個主要城市
JPcitys = pd.read_csv('C:/Users/sandr/DS/japanmaincity-47final.csv')

#rows = list()

for i in range(JPcitys.shape[0]):
#for i in range(12,13):
    city_area = JPcitys.iloc[i][0]
    city_area_code = JPcitys.iloc[i][1]
    city_name = JPcitys.iloc[i][2]
    city_name_code = JPcitys.iloc[i][3]
    
    rows = list()

    # 抓取該城市資料2000/1-2023/4
    for yyyy in range(2000,2024):
        for mm in range(1,13):
            if yyyy == 2023:
                if mm >= 5:
                    break

            html = urllib.request.urlopen("https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no="+str(city_area_code)+"&block_no="+str(city_name_code)+"&year="+str(yyyy)+"&month="+twobyte(mm)+"&day=&view=").read()
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find('table', {'id': 'tablefix1'})
            table.select("tr")

            trs = table.find_all('tr')[4:]
            for tr in trs:
                row = [td.text.replace('\n', '') for td in tr.find_all('td')]
                yyyymm = str(yyyy) + twobyte(mm)
                row.insert(0, yyyymm)
                rows.append(row)
            
            time.sleep(1)
            

    df = pd.DataFrame(data=rows, columns=columns_t)
    df

    #存成csv檔
    num = i + 1
    outputpath = twobyte(num) + city_name + '_200001-202304.csv'
    df.to_csv(outputpath,sep=',',index=False,header=True, encoding='utf_8_sig') 
    print(twobyte(num) + city_name + "輸出成功")